# <center> Implementing Decision Trees </center>
## <center> INF283 - Project 1 </center>
### <center> Sindre E. de Lange </center>

In [1]:
import numpy as np
import os
import pandas as pd

## Getting data in order to test the model

In [4]:
# ! git clone https://github.com/sjwhitworth/golearn.git

Cloning into 'golearn'...


In [2]:
DATASET_PATH = "golearn/examples/datasets/"
print(os.listdir(DATASET_PATH))

['articles.csv', 'c45-numeric.csv', 'chim.csv', 'exam.csv', 'exams.csv', 'house-votes-84.csv', 'iris.arff', 'iris.csv', 'iris_binned.csv', 'iris_headers.csv', 'iris_headers_subset.csv', 'iris_sorted_asc.csv', 'iris_sorted_desc.csv', 'mnist_test.csv', 'mnist_train.csv', 'randomdata.csv', 'sources.txt', 'tennis.csv', 'weather.arff']


In [3]:
tennis_dataset = "tennis.csv"
dataset_tennis = pd.read_csv(DATASET_PATH + tennis_dataset)

In [4]:
dataset_tennis.head()

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


In [5]:
dataset_tennis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 5 columns):
outlook     14 non-null object
temp        14 non-null object
humidity    14 non-null object
windy       14 non-null bool
play        14 non-null object
dtypes: bool(1), object(4)
memory usage: 542.0+ bytes


In [6]:
dataset_tennis.describe()

,outlook,temp,humidity,windy,play
count,14,14,14,14,14
unique,3,3,2,2,2
top,sunny,mild,high,False,yes
freq,5,6,7,8,9


In [254]:
data_enc = dataset_tennis
for columns in dataset_tennis:
    data_enc[columns], unique_data = pd.factorize(dataset_tennis[columns])

In [255]:
print(data_enc.head())

   outlook  temp  humidity  windy  play
0        0     0         0      0     0
1        0     0         0      1     0
2        1     0         0      0     1
3        2     1         0      0     1
4        2     2         1      0     1


In [263]:
X_enc = data_enc.drop(['play'], axis=1)
y_enc = data_enc['play']

In [264]:
X_enc.head()

,outlook,temp,humidity,windy
0,0,0,0,0
1,0,0,0,1
2,1,0,0,0
3,2,1,0,0
4,2,2,1,0


In [258]:
y_enc.head()

0    0
1    0
2    1
3    1
4    1
Name: play, dtype: int64

In [259]:
from sklearn import tree
from sklearn.model_selection import train_test_split
clf = tree.DecisionTreeClassifier()
X_train, X_test, y_train, y_test = train_test_split(X_enc, y_enc, test_size=0.3, random_state=42)
clf = clf.fit(X_train, y_train)
print(clf.score(X_test, y_test))

0.6


## Data for verifying the model *check*

# Model implementation

## 1. Implement the ID3 algorithm from scratch

In [265]:
def calc_entropy(p):
    """Calculate the entropy for a given fraction"""
    if p!=0:
        return -p * np.log2(p)
    else:
        return 0

In [266]:
def calc_entropy_system(target_variable):
    """Calculates the entropy of the system
    Data is the target variable"""
    tot_len = len(target_variable)
    unique, counts = np.unique(target_variable, return_counts=True)
    dic = dict(zip(unique, counts))
    entropy = 0
    for key, value in dic.items():
        entropy += calc_entropy(value/tot_len)
    return entropy

In [262]:
type(data_enc[['outlook', 'play']])

pandas.core.frame.DataFrame

In [253]:
df = pd.DataFrame()
df = pd.concat([X_enc, y_enc], axis=1)
print(df)

    outlook  temp  humidity  windy  play
0         0     0         0      0     0
1         0     0         0      1     0
2         1     0         0      0     1
3         2     1         0      0     1
4         2     2         1      0     1
5         2     2         1      1     0
6         1     2         1      1     1
7         0     1         0      0     0
8         0     2         1      0     1
9         2     1         1      0     1
10        0     1         1      1     1
11        1     1         0      1     1
12        1     0         1      0     1
13        2     1         0      1     0


In [279]:
import collections

def calc_entropy_dataset(X, y):
    tot_num_occurences = len(y)
    entropy_system = calc_entropy_system(y)
    # Small hack because the system was initially made for full Dataframe input
    data = pd.concat([X, y], axis=1)
    X_y_zip = {}
    for columns in X:
        # Map each value in each column to their "outcome"/target variable
        frames = []
        X_y_zip[columns] = data[[columns, y.name]].apply(tuple, axis=1)
    each_feature_w_entropy = calc_entropy_feature(X_y_zip, tot_num_occurences)
    

    return calc_entropy_all_branches(each_feature_w_entropy, entropy_system)

In [280]:
def calc_entropy_feature(X_y_zip, tot_num_occurences):
    """Calcalutes the entropy for each feature in a dataset.
    Assumes that all unique columns are zipped with the target variable.
    Returns a dictionary on the format 
        {'column feature': 
            {unique value: [
                number of times this value occured in the set
                number of values in the set
                entropy when this value occured in the set
            ], ... }}"""
    columns_entropy = {}
    for feature in X_y_zip:
        # Get unique variables for each key
        list_of_unique_variables = list(set([x[0] for x in X_y_zip[feature].values]))

        val_dict = {}
        for val in list_of_unique_variables:
            # Total number of days for each unique variable
            num_days_val = len([x[1] for x in X_y_zip[feature] if x[0] == val])
            # Total number of days for each key (assuming it is binary (and tennis), for now)
            num_days_val_tennis = len([x[1] for x in X_y_zip[feature] if x[0] == val and x[1] == 1])
            num_days_val_not_tennis = num_days_val - num_days_val_tennis
            #Calculate entropy for each unique value
            val_entropy = calc_entropy(num_days_val_tennis/num_days_val) + calc_entropy(num_days_val_not_tennis/num_days_val)
            # Make a list with relevant data for each unique value
            val_list = [num_days_val, tot_num_occurences, val_entropy]
            # Append that list to a dictionary, where the unique value is key
            val_dict[val] = val_list
        # Append dictionaries for unique values, to their respectively feature
        columns_entropy[feature] = val_dict
    return columns_entropy

In [281]:
def calc_entropy_all_branches(feature_entr_dict, entropy_src):
    """Calculates the entropy among all the branches
    Expects a dictionary on the format 
        {'column feature': 
            {unique value: [
                number of this value occured in the set
                number of values in the set
                entropy when this value occured in the set
            ], ... }}"""
    column_entropy_dict_full = {}
    for column_feature in feature_entr_dict:
        entropy_all = 0
        # Take each value from each 'unique value', for each 'column feature' from the inputed dictionary
        # and calculates the entropy for each 'unique value', e.g. sunny, rainy, etc. 
        for unique_val in feature_entr_dict[column_feature]:
            # NOTE: As mentioned in PyDoc - assumes this format
            num_val = feature_entr_dict[column_feature][unique_val][0]
            num_tot = feature_entr_dict[column_feature][unique_val][1]
            num_val_entropy = feature_entr_dict[column_feature][unique_val][2]
            entropy_all += (num_val/num_tot)*num_val_entropy
        column_entropy_dict_full[column_feature] = entropy_all
    
    information_gain_dict = {}
    for key, value in column_entropy_dict_full.items():
        information_gain_dict[key] = randomness_reduction(entropy_src, value)
    return information_gain_dict

In [282]:
def randomness_reduction(entropy_src, entropy_branch):
    """ Calculates the reduction in randomness, aka Information Gain.
    Takes in the entropy of the entire system, and the entropy for one branch
    Returns the Information Gain - restricted to 3 decimals."""
    return (round(entropy_src - entropy_branch, 3))

In [285]:
def getLargestInformationGain(X, y):
    """Gets the largest IG for any given dataset (that is Pandas DataFrame)"""
    ig_dict = calc_entropy_dataset(X, y)
    return (max(ig_dict, key=ig_dict.get))

In [286]:
getLargestInformationGain(X_enc, y_enc)

'outlook'

# Notes:
- The best feature to pick as the one to classify on is the one with the most information (gain), i.e. highest entropy
    - After finding the best feature, re-evaluate the entropy of each feature and again pick the one with the highest entropy

In [302]:
# Utilizing the simple tree library that is "Treelib"
from treelib import Node, Tree

def learn(X, y, impurity_measure='entropy'):
    """Function that learns a decision tree classifier from data.
        Default impurity measure for information gain is Entropy."""

    tree = Tree()
    current_node = None
    current_node_children = None
    tree_depth = 0
    for feature in X:            
        # Check if tree is empty --> define the root
        if tree.size() == 0:
            print("Initalizing the tree with a root node = highest IG")
            root_node_name = getLargestInformationGain(X, y)
            tree.create_node(tag=root_node_name, identifier=root_node_name)
            current_node = tree.get_node(root_node_name)
        # Root node is initialized - add the other features
        else:
            break
            #print("Initializing the feature: " + feature + " as a branch to: " + str(current_node))
            #new_node_name = getLargestInformationGain(X, y)
            # Initialize the new feature to the previous nodes' children
            #for children in current_node_children:
            #    print("Appending the " + feature + " to the children of " + str(current_node))
            #    print("Children: " + str(children))
            #    tree.create_node(tag=new_node_name, parent=children)
            #    print(tree.show())
            #current_node = tree.get_node(next_node_name)
        
        # Create children for each possible value - e.g. 0, 1, 2
        for value in set(X[feature]):
            # Get the relevant rows to store in the children node
            # E.g. the rows where the feature's value is this value
            relevant_rows_df = pd.concat([X, y], axis=1).loc[X[feature] == value]
            children_node_name = feature + ":" + str(value)
            tree.create_node(tag=children_node_name, parent=current_node, data=relevant_rows_df)
        
        # Get ready for next feature
        X = X.drop([feature], axis=1)
        current_node_children = current_node.fpointer
        
    
        print(tree.show())

In [303]:
learn(X_enc, y_enc)

Initalizing the tree with a root node = highest IG
outlook
├── outlook:0
├── outlook:1
└── outlook:2

None


In [25]:
def predict(x, tree):
    """Predict class label of some new data point x."""

Create a tree: (1:16:11, https://www.youtube.com/watch?v=3jl2h9hSRvc&feature=youtu.be)

In [26]:
# https://stackoverflow.com/questions/41760856/most-simple-tree-data-structure-in-python-that-can-be-easily-traversed-in-both
class Tree(object):
    def __init__(self, data, children=None, parent=None):
        self.data = data
        self.children = {}
        self.parent = parent

    def add_child(self, data):
        new_child = Tree(data, parent=self)
        self.children.append(new_child)
        return new_child

    def is_root(self):
        return self.parent is None

    def is_leaf(self):
        return not self.children

    """ ToString method """
    def __str__(self):
        if self.is_leaf():
            return str(self.data)
        return '{data} [{children}]'.format(data=self.data, children=', '.join(map(str, self.children)))

Uncomment if necessary to install treelib

In [27]:
# ! pip install treelib

In [28]:
# Utilizing the simple tree library that is "Treelib"
from treelib import Node, Tree

In [29]:
tree = Tree()
root_node_name = getLargestInformationGain(dataset_tennis, 'play')
tree.create_node(tag=root_node_name, identifier=root_node_name)

Node(tag=outlook, identifier=outlook, data=None)

In [30]:
tree.create_node(identifier=root_node_name + '0', parent=root_node_name)
tree.create_node(identifier=root_node_name + '1', parent=root_node_name)
tree.create_node(identifier=root_node_name + '2', parent=root_node_name)

Node(tag=outlook2, identifier=outlook2, data=None)

In [31]:
print(tree.show())

outlook
├── outlook0
├── outlook1
└── outlook2

None


In [32]:
dataset_rem_outlook = dataset_tennis.drop(['outlook'], axis=1)

In [33]:
dataset_rem_outlook.head()

,temp,humidity,windy,play
0,0,0,0,0
1,0,0,1,0
2,0,0,0,1
3,1,0,0,1
4,2,1,0,1


In [34]:
next_node_name = getLargestInformationGain(dataset_rem_outlook, 'play')

In [35]:
tree.create_node(identifier=next_node_name + '00', parent=root_node_name + '0')
tree.create_node(identifier=next_node_name + '10', parent=root_node_name + '0')
tree.create_node(identifier=next_node_name + '20', parent=root_node_name + '0')

Node(tag=humidity20, identifier=humidity20, data=None)

In [36]:
tree.create_node(identifier=next_node_name + '01', parent=root_node_name + '1')
tree.create_node(identifier=next_node_name + '11', parent=root_node_name + '1')
tree.create_node(identifier=next_node_name + '21', parent=root_node_name + '1')

Node(tag=humidity21, identifier=humidity21, data=None)

In [37]:
tree.create_node(identifier=next_node_name + '02', parent=root_node_name + '2')
tree.create_node(identifier=next_node_name + '12', parent=root_node_name + '2')
tree.create_node(identifier=next_node_name + '22', parent=root_node_name + '2')

Node(tag=humidity22, identifier=humidity22, data=None)

In [38]:
print(tree.show())

outlook
├── outlook0
│   ├── humidity00
│   ├── humidity10
│   └── humidity20
├── outlook1
│   ├── humidity01
│   ├── humidity11
│   └── humidity21
└── outlook2
    ├── humidity02
    ├── humidity12
    └── humidity22

None
